<a href="https://colab.research.google.com/github/KimRass/Programming/blob/master/Data%20Science/Machine%20Learning/NLP/Portuguese-English%20%26%20Transformer%20(NMT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modeling" data-toc-modified-id="Modeling-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Inference" data-toc-modified-id="Inference-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Inference</a></span></li></ul></div>

In [ ]:
# Data Science/Machine Learning/NLP/Portuguese-English & Transformer (NMT).ipynb
# !pip install -q tf-nightly

In [88]:
from google.colab import drive
import os
drive.mount("/content/drive")
os.chdir("/content")

import time
import tensorflow_datasets as tfds
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer, Dense, Flatten, Dropout, Concatenate, Add, Dot, Multiply, Reshape, Activation, BatchNormalization, LayerNormalization, SimpleRNNCell, RNN, SimpleRNN, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D, Conv2D, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, ZeroPadding2D, RepeatVector
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adagrad, Adam
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
# MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras import losses
# MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import linear, sigmoid, relu
from tensorflow.keras.initializers import RandomNormal, glorot_uniform, he_uniform, Constant

plt.style.use("dark_background")

Mounted at /content/drive


In [89]:
# `with_info`: If `True`, `tfds.load()` will return the tuple `(tf.data.Dataset, tfds.core.DatasetInfo)`, the latter containing the info associated with the builder.
# `as_supervised`: If `True`, the returned `tf.data.Dataset` will have a 2-tuple structure `(input, label)` according to `builder.info.supervised_keys`. If `False`, the returned `tf.data.Dataset` will have a dictionary with all the features.
dataset, metadata = tfds.load("ted_hrlr_translate/pt_to_en", with_info=True, as_supervised=True)
dataset_tr = dataset["train"]
dataset_val = dataset["validation"]
dataset_te = dataset["test"]

In [90]:
tokenizer_src = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus((pt.numpy() for pt, en in dataset_tr), target_vocab_size=2**13)
tokenizer_tar = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus((en.numpy() for pt, en in dataset_tr), target_vocab_size=2**13)

In [91]:
max_len = 40
buffer_size = tf.data.experimental.cardinality(dataset_tr).numpy()
batch_size = 64

def encode(lang1, lang2):
    lang1 = [tokenizer_src.vocab_size] + tokenizer_src.encode(lang1.numpy()) + [tokenizer_src.vocab_size + 1]
    lang2 = [tokenizer_tar.vocab_size] + tokenizer_tar.encode(lang2.numpy()) + [tokenizer_tar.vocab_size + 1]
    return lang1, lang2

def tf_encode(pt, en):
    # `func`: A Python function that accepts `inp` as arguments, and returns a value (or list of values) whose type is described by `Tout`.
    # `inpt`: Input arguments for func. A list whose elements are Tensors or a single Tensor.
    result_pt, result_en = tf.py_function(func=encode, inp=[pt, en], Tout=[tf.int64, tf.int64])
    result_pt.set_shape([None])
    result_en.set_shape([None])
    return result_pt, result_en

def filter_max_len(x, y):
    return tf.logical_and(tf.size(x) <= max_len, tf.size(y) <= max_len)

# This transformation applies `map_func` to each element of this dataset, and returns a new dataset containing the transformed elements, in the same order as they appeared in the input. `map_func` can be used to change both the values and the structure of a dataset's elements.
dataset_tr = dataset_tr.map(tf_encode)
# `predicate`: A function mapping a dataset element to a boolean.
# Returns the dataset containing the elements of this dataset for which `predicate` is `True`.
dataset_tr = dataset_tr.filter(filter_max_len)
# The first time the dataset is iterated over, its elements will be cached either in the specified file or in memory. Subsequent iterations will use the cached data.
# `filename`: When caching to a file, the cached data will persist across runs. Even the first iteration through the data will read from the cache file. Changing the input pipeline before the call to `cache()` will have no effect until the cache file is removed or the `filename` is changed. If a `filename` is not provided, the dataset will be cached in memory.
dataset_tr = dataset_tr.cache()
# For perfect shuffling, a `buffer_size` greater than or equal to the full size of the dataset is required.
# If not, only the first `buffer_size` elements will be selected randomly.
# `reshuffle_each_iteration` controls whether the shuffle order should be different for each epoch.
dataset_tr = dataset_tr.shuffle(buffer_size)
# Pad to the smallest per-`batch size` that fits all elements.
# Unlike `batch()`, the input elements to be batched may have different shapes, and this transformation will pad each component to the respective shape in `padded_shapes`. The `padded_shapes` argument determines the resulting shape for each dimension of each component in an output element.
# `padded_shapes`:
    # If `None`: The dimension is unknown, the component will be padded out to the maximum length of all elements in that dimension.
    # If not `None`: The dimension is a constant, the component will be padded out to that length in that dimension.
# `padding_values`
# `drop_remainder`
dataset_tr = dataset_tr.padded_batch(batch_size)
# Most dataset input pipelines should end with a call to prefetch. This allows later elements to be prepared while the current element is being processed. This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.
# `buffer_size`: The maximum number of elements that will be buffered when prefetching. If the value `tf.data.AUTOTUNE` is used, then the buffer size is dynamically tuned.
dataset_tr = dataset_tr.prefetch(tf.data.AUTOTUNE)

dataset_val = dataset_val.map(tf_encode)
dataset_val = dataset_val.filter(filter_max_len)
dataset_val = dataset_val.padded_batch(batch_size)

# Modeling

In [92]:
n_layers = 4
d_model = 128
dff = 512
n_heads = 8
dk = d_model//n_heads

def padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    # `(batch_size, 1, 1, seq_len)`
    return seq[:, None, None, :]

def look_ahead_mask(tar):
    size = tf.shape(tar)[1]
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    p_mask = padding_mask(tar)
    return tf.maximum(p_mask, mask)

def positional_encoding_matrix():
    a, b = np.meshgrid(np.arange(d_model), np.arange(max_len))
    pe_mat = b/10000**(2*(a//2)/d_model)
    pe_mat[:, 0::2] = np.sin(pe_mat[:, 0::2])
    pe_mat[:, 1::2] = np.cos(pe_mat[:, 1::2])
    pe_mat = pe_mat[None, ...]
    pe_mat = tf.cast(pe_mat, dtype=tf.float32)
    return pe_mat

# `x`: `(batch_size, seq_len)`
class Embedder(Layer):
    def __init__(self, vocab_size):
        super().__init__()
        
        self.embedding = Embedding(input_dim=vocab_size, output_dim=d_model)
        self.dropout = Dropout(rate=0.1)
        self.pe_mat = positional_encoding_matrix()
        
    def __call__(self, x, training):
        # `(batch_size, seq_len, d_model)`
        z = self.embedding(x)

        # Positional Encoding
        seq_len = tf.shape(x)[1]
        z = (d_model**0.5)*z + self.pe_mat[:, :seq_len, :]
        z = self.dropout(z, training=training)
        # `(batch_size, seq_len, d_model)`
        return z

class MultiheadAttention(Layer):
    def __init__(self):
        super().__init__()

        self.dense_q = Dense(units=d_model)
        self.dense_k = Dense(units=d_model)
        self.dense_v = Dense(units=d_model)
        self.dense = Dense(units=d_model)
        
    def __call__(self, queries, keys, values, mask):
        batch_size = tf.shape(queries)[0]

        def split_heads(x):
            x = tf.reshape(x, shape=(batch_size, -1, n_heads, dk))
            return tf.transpose(x, perm=[0, 2, 1, 3])

        # `(batch_size, seq_len_dec, d_model)`
        queries = self.dense_q(queries)
        # `(batch_size, seq_len_enc, d_model)`
        keys = self.dense_k(keys)
        # `(batch_size, seq_len_enc, d_model)`
        values = self.dense_v(values)

        # `(batch_size, n_heads, seq_len_dec, dk)`
        queries = split_heads(queries)
        # `(batch_size, n_heads, seq_len_enc, dk)`
        keys = split_heads(keys)
        # `(batch_size, n_heads, seq_len_enc, dk)`
        values = split_heads(values)

        # Scaled Dot-Product Attention
        # `(batch_size, n_heads, seq_len_dec, seq_len_enc)`
        attn_scores = tf.matmul(queries, keys, transpose_b=True)/dk**0.5
        if mask is not None:
            attn_scores = attn_scores + (mask*-1e9)
        attn_weights = tf.nn.softmax(attn_scores, axis=-1)
        # `(batch_size, n_heads, seq_len_dec, dk)`
        context_vec = tf.matmul(attn_weights, values)

        # `(batch_size, seq_len_dec, n_heads, dk)`
        z = tf.transpose(context_vec, perm=[0, 2, 1, 3])
        # `(batch_size, seq_len_dec, d_model)`
        z = tf.reshape(z, shape=(batch_size, -1, d_model))
        z = self.dense(z)
        return z, attn_weights

class PointwiseFFNN(Layer):
    def __init__(self):
        super().__init__()
        
        self.dense1 = Dense(units=dff, activation="relu")
        self.dense2 = Dense(units=d_model)
        
    def __call__(self, x):
        # `(batch_size, seq_len, dff)`
        z = self.dense1(x)
        # `(batch_size, seq_len, d_model)`
        z = self.dense2(z)
        return z

# ENCODER
class Encoder(Layer):
    def __init__(self):
        super().__init__()
        
        self.mha = MultiheadAttention()
        self.layernormalization1 = LayerNormalization(epsilon=1e-6)
        self.ffnn = PointwiseFFNN()
        self.layernormalization2 = LayerNormalization(epsilon=1e-6)
        
    def __call__(self, x, training, padding_mask):
        ## "Self-Attention" Part
        z1, _ = self.mha(queries=x, keys=x, values=x, mask=padding_mask) 
        z1 = Dropout(rate=0.1)(z1, training=training)
        ## "Add & Normalize" Part
        z1 = x + z1
        z1 = self.layernormalization1(z1)
        ## "Feed Forward" Part
        z2 = self.ffnn(z1)
        z2 = Dropout(rate=0.1)(z2, training=training)
        ## "Add & Normalize" Part
        z2 = z1 + z2
        z2 = self.layernormalization2(z2)
        # (batch_size, seq_len_enc, d_model)
        return z2

# DECODER
class Decoder(Layer):
    def __init__(self):
        super().__init__()
        
        self.mha1 = MultiheadAttention()
        self.layernormalization1 = LayerNormalization(epsilon=1e-6)
        self.mha2 = MultiheadAttention()
        self.layernormalization2 = LayerNormalization(epsilon=1e-6)
        self.ffnn = PointwiseFFNN()
        self.layernormalization3 = LayerNormalization(epsilon=1e-6)
        
    def __call__(self, x, enc_output, training, padding_mask, look_ahead_mask):
        ## "Self-Attention" Part
        z1, attn_weights1 = self.mha1(queries=x, keys=x, values=x, mask=look_ahead_mask)
        z1 = Dropout(rate=0.1)(z1, training=training)
        ## "Add & Normalize" Part
        z1 = x + z1
        z1 = self.layernormalization1(z1)
        ## "Encoder-Decoder Attention" Part
        z2, attn_weights2 = self.mha2(queries=z1, keys=enc_output, values=enc_output, mask=padding_mask)
        z2 = Dropout(rate=0.1)(z2, training=training)
        ## "Add & Normalize" Part
        z2 = z1 + z2
        z2 = self.layernormalization2(z2)
        ## "Feed Forward" Part
        z3 = self.ffnn(z2)
        z3 = Dropout(rate=0.1)(z3, training=training)
        ## "Add & Normalize" Part
        z3 = z2 + z3
        # (batch_size, seq_len_dec, d_model)
        z3 = self.layernormalization3(z3)
        return z3, attn_weights1, attn_weights2

class Transformer(Model):
    def __init__(self):
        super().__init__()
        
        self.embedder_enc = Embedder(vocab_size=tokenizer_src.vocab_size + 2)
        # self.embedder_enc = TransformerEmbedding(vocab_size=tokenizer_src.vocab_size + 2)
        self.encoder = Encoder()
        self.embedder_dec = Embedder(vocab_size=tokenizer_tar.vocab_size + 2)
        # self.embedder_dec = TransformerEmbedding(vocab_size=tokenizer_tar.vocab_size + 2)
        self.decoder = Decoder()
        self.dense = Dense(units=tokenizer_tar.vocab_size + 2)
        
    def __call__(self, enc, dec, training):
        enc_p_mask = padding_mask(enc)
        dec_p_mask = padding_mask(enc)
        l_mask = look_ahead_mask(dec)

        z_enc = self.embedder_enc(enc, training=training)
        for _ in range(n_layers):
            z_enc = self.encoder(z_enc, training=training, padding_mask=enc_p_mask)

        z_dec = self.embedder_dec(dec, training=training)
#         dic = dict()
        for i in range(n_layers):
            z_dec, attn_weights1, attn_weights2 = self.decoder(z_dec, z_enc, training=training, padding_mask=dec_p_mask, look_ahead_mask=l_mask)
#             dic[f"self_attention_attn_weights_{i}"] = attn_weights1
#             dic[f"encoder_decoder_attention_attn_weights_{i}"] = attn_weights2

        outputs = self.dense(z_dec)
        return outputs

# Training

In [93]:
model = Transformer()

class CustomSchedule(LearningRateSchedule):
    def __init__(self, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        return (d_model**-0.5)*tf.math.minimum(step**-0.5, step*(self.warmup_steps**-1.5))

learning_rate = CustomSchedule()
optimizer = Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# Loss function
loss_obj = losses.SparseCategoricalCrossentropy(from_logits=True, reduction="none")
def loss_func(y_true, y_pred):
    # `True` if not `0` else `False`
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))
    # `from_logits=True`: Computes loss from a logit (i.e, value in [int, int])
    # `from_logits=False`: Computes loss from a probability (i.e, value in [0, 1])
    # `reduction="none"`: 계산된 Losses에 대해 별도의 연산을 하지 않습니다.
    # `reduction="auto"`: Batch에 대해 평균을 출력합니다.
    # `reduction="sum"`: Batch에 대해 합을 출력합니다.
    loss = loss_obj(y_true, y_pred)
    # `1` if True else `0`
    mask = tf.cast(mask, dtype=tf.float32)
    loss *= mask
    return tf.math.reduce_sum(loss)/tf.math.reduce_sum(mask)

def acc_func(real, pred):
    accuracies = tf.equal(real, tf.argmax(pred, axis=2))

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    accuracies = tf.math.logical_and(mask, accuracies)

    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

tr_loss = metrics.Mean(name="train_loss")
tr_acc = metrics.Mean(name="train_accuracy")

@tf.function(input_signature=[tf.TensorSpec(shape=(None, None), dtype=tf.int64), tf.TensorSpec(shape=(None, None), dtype=tf.int64)])
def train_step(enc, dec):
    dec_input = dec[:, :-1]
    dec_true = dec[:, 1:]
    with tf.GradientTape() as tape:
        dec_pred = model(enc=enc, dec=dec_input, training=True)
        loss = loss_func(dec_true, dec_pred)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    tr_loss(loss)
    tr_acc(acc_func(dec_true, dec_pred))

ckpt_path = "drive/MyDrive/NLP/pt_to_en_transformer"
ckpt = tf.train.Checkpoint(optimizer=optimizer, model=model)
ckpt_manager = tf.train.CheckpointManager(ckpt, directory=ckpt_path, max_to_keep=5)
if ckpt_manager.latest_checkpoint:
    ckpt.restore(save_path=ckpt_manager.latest_checkpoint)
    print ("Latest checkpoint restored!")

In [ ]:
epochs = 24
for epoch in range(1, epochs + 1):
    start = time.time()
    tr_loss.reset_states()
    tr_acc.reset_states()
    for (batch, (src, tar)) in enumerate(dataset_tr):
        train_step(src, tar)

        if batch%50 == 0:
            print(f"Epoch: {epoch:3d} | Batch: {batch:5d} | Loss: {tr_loss.result():5.4f} | Accuracy: {tr_acc.result():5.4f}")

    if epoch%1 == 0:
        # Every time `ckpt_manager.save()` is called, `save_counter` is increased.
        # `save_path`: The path to the new checkpoint. It is also recorded in the `checkpoints` and `latest_checkpoint` properties. `None` if no checkpoint is saved.
        save_path = ckpt_manager.save()
        print (f"Saving checkpoint for epoch {epoch} at {save_path}")
        print(f"Epoch: {epoch:3d} | Loss: {tr_loss.result():5.4f} | Accuracy: {tr_acc.result():5.4f}")
        print (f"Time taken for 1 epoch: {time.time() - start:5.0f} secs\n")

Epoch:   1 | Batch:     0 | Loss: 9.0148 | Accuracy: 0.0000
Epoch:   1 | Batch:    50 | Loss: 8.9731 | Accuracy: 0.0004
Epoch:   1 | Batch:   100 | Loss: 8.8829 | Accuracy: 0.0175
Epoch:   1 | Batch:   150 | Loss: 8.7739 | Accuracy: 0.0292
Epoch:   1 | Batch:   200 | Loss: 8.6456 | Accuracy: 0.0351
Epoch:   1 | Batch:   250 | Loss: 8.4917 | Accuracy: 0.0398
Epoch:   1 | Batch:   300 | Loss: 8.3174 | Accuracy: 0.0464
Epoch:   1 | Batch:   350 | Loss: 8.1324 | Accuracy: 0.0550
Epoch:   1 | Batch:   400 | Loss: 7.9519 | Accuracy: 0.0635
Epoch:   1 | Batch:   450 | Loss: 7.7896 | Accuracy: 0.0701
Epoch:   1 | Batch:   500 | Loss: 7.6470 | Accuracy: 0.0760
Epoch:   1 | Batch:   550 | Loss: 7.5194 | Accuracy: 0.0812
Epoch:   1 | Batch:   600 | Loss: 7.4000 | Accuracy: 0.0874
Epoch:   1 | Batch:   650 | Loss: 7.2873 | Accuracy: 0.0944
Epoch:   1 | Batch:   700 | Loss: 7.1774 | Accuracy: 0.1011
Saving checkpoint for epoch 1 at drive/MyDrive/NLP/pt_to_en_transformer/ckpt-1
Epoch:   1 | Loss: 7.

# Inference

In [ ]:
def evaluate(src_sentence):
    # src_sentence : 문자 (string)
    bos = [tokenizer_src.vocab_size]
    eos = [tokenizer_src.vocab_size + 1]

    src_sentence = bos + tokenizer_src.encode(src_sentence) + eos
    encoder_input = tf.expand_dims(src_sentence, 0)

    decoder_input = [tokenizer_tar.vocab_size]
    output = tf.expand_dims(decoder_input, 0)
    for i in range(max_len):
        # (batch_size, seq_len, vocab_size)
        predictions = model(encoder_input, output, False)
        # 예측 결과에서 마지막 부분만 추출
        # (batch_size, 1, vocab_size)
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        if predicted_id == tokenizer_tar.vocab_size + 1:
            return tf.squeeze(output, axis=0)
        # 예측된 단어를 전 단어와 결합하여 다음 예측에 써먹음
        output = tf.concat([output, predicted_id], axis=-1)
        return tf.squeeze(output, axis=0)

def translate(sentence):
    result= evaluate(sentence)
    predicted_sentence = tokenizer_tar.decode([i for i in result if i < tokenizer_tar.vocab_size])  

    print("input: {}".format(sentence))
    print("Predicted translation: {}".format(predicted_sentence))

In [ ]:
translate("este é um problema que temos que resolver.")
print("Real translation: this is a problem we have to solve .")

input: este é um problema que temos que resolver.
Predicted translation: and 
Real translation: this is a problem we have to solve .


In [ ]:
translate("os meus vizinhos ouviram sobre esta ideia.")
print("Real translation: and my neighboring homes heard about this idea .")

input: os meus vizinhos ouviram sobre esta ideia.
Predicted translation: and 
Real translation: and my neighboring homes heard about this idea .


In [ ]:
translate("vou então muito rapidamente partilhar convosco algumas histórias de algumas coisas mágicas que aconteceram.")
print("Real translation: so i 'll just share with you some stories very quickly of some magical things that have happened .")

input: vou então muito rapidamente partilhar convosco algumas histórias de algumas coisas mágicas que aconteceram.
Predicted translation: but 
Real translation: so i 'll just share with you some stories very quickly of some magical things that have happened .


In [ ]:
translate("este é o primeiro livro que eu fiz.")
print ("Real translation: this is the first book i've ever done.")

input: este é o primeiro livro que eu fiz.
Predicted translation: and 
Real translation: this is the first book i've ever done.


출처  
http://jalammar.github.io/illustrated-gpt2/  
https://d2l.ai/chapter_recurrent-modern/seq2seq.html  
https://www.tensorflow.org/tutorials/text/transformer
